# __라이브러리 로드__

In [1]:
import pandas as pd
import numpy as np
from numpy.random import RandomState
import re
import torch
from tqdm import tqdm
from konlpy.tag import Okt
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

/home/hylee/miniconda3/envs/p3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#number of comments
NOC = 50
#k = 0 #start idx

## __1. 데이터 크롤링__

## __2. 데이터 전처리__

> __2.1 크롤링 데이터 로드__
- 로드 후 일단 10개만 붙이기

In [3]:
df = pd.read_excel("youtube_replies.xlsx")
#cols = list(df.keys())[1:]
df_trans = df.T[1:].reset_index()
df_trans.rename(columns={'index':'view'}, inplace=True)
df_trans.iloc[:1,:3] #example

,view,0,1
0,194914,이 교수님 정말 재미있게 강의해주시네요... 자주 뵈었음 좋겠습니다.,김기범 교수님 쓸데없는 은유나 비유없이 요점만 명확히 설명 해 주셔서 이해도 ...


In [4]:
new_df = pd.DataFrame([])
new_df["Views"] = df_trans["view"]
new_df["comments"] = df_trans.iloc[:,1]

In [5]:
new_df = pd.DataFrame([])
new_df["comments"] = df_trans.iloc[:,1]
for i in range(2,NOC+1): 
    new_df["comments"] += "\n" + df_trans.iloc[:,i]
new_df["views"] = df_trans["view"]
new_df = new_df.dropna() 
new_df

,comments,views
0,이 교수님 정말 재미있게 강의해주시네요... 자주 뵈었음 좋겠습니다.\n김기범 교수...,194914
1,OCN에 켠김에 왕까지에서 허준씨를 처음 봤을때는 이렇게 될 줄은 상상도 못 했습니...,319933
3,교수님 강의를 이렇게 쉽게 들을 수 있다니 행운이고 감사합니다.\n학창시절땐 과학이...,98384
4,영화 <크리에이터> 예매권 이벤트 이 영상에 영화에 대한 기대평 댓글을\n남겨주신...,402196
5,우와 영상 재밌게봤어요! 넘넘 차분하게 전달력 있게 말씀해주시네요.. 하도 슈뢰딩거...,150218
...,...,...
195,다양한 천문학 이야기 전할 수 있는 좋은 기회 주신 BODA 분들께 감사드립니다! ...,287723
196,곧 안전하고 값싼 우주 여행이 실현되길 바랍니다 우주 여행 관련 이야기할 수 있는...,2298575
197,오늘도 초대해 주셔서 감사합니다!!!! 자주 놀러와서 축구 이야기 해볼게요!!영상 ...,362675
198,"여자를 위해, 여자를 가장 잘 아는 산부인과의사언니 김지연 원장이 개발한 우먼스37...",353262


> __2.2 데이터 전처리__

In [6]:
#punctuation 제거, 특수문자 제거, lowercasing
def remove_white_space(text):
    text = re.sub(r'[\t\r\n\f\v]', ' ', str(text))
    return text

def remove_special_char(text):
    text = re.sub('[^ ㄱ-ㅣ가-힣 0-9 a-z A-Z]+', ' ', str(text))
    text.lower()
    return text

new_df.comments = new_df.comments.apply(remove_white_space)
new_df.comments = new_df.comments.apply(remove_special_char)
new_df

,comments,views
0,이 교수님 정말 재미있게 강의해주시네요 자주 뵈었음 좋겠습니다 김기범 교수님 ...,194914
1,OCN에 켠김에 왕까지에서 허준씨를 처음 봤을때는 이렇게 될 줄은 상상도 못 했습니...,319933
3,교수님 강의를 이렇게 쉽게 들을 수 있다니 행운이고 감사합니다 학창시절땐 과학이나...,98384
4,영화 크리에이터 예매권 이벤트 이 영상에 영화에 대한 기대평 댓글을 남겨주신 ...,402196
5,우와 영상 재밌게봤어요 넘넘 차분하게 전달력 있게 말씀해주시네요 하도 슈뢰딩거고...,150218
...,...,...
195,다양한 천문학 이야기 전할 수 있는 좋은 기회 주신 BODA 분들께 감사드립니다 ...,287723
196,곧 안전하고 값싼 우주 여행이 실현되길 바랍니다 우주 여행 관련 이야기할 수 있는...,2298575
197,오늘도 초대해 주셔서 감사합니다 자주 놀러와서 축구 이야기 해볼게요 영상 재미있게...,362675
198,여자를 위해 여자를 가장 잘 아는 산부인과의사언니 김지연 원장이 개발한 우먼스37...,353262


> __2.3 토크나이징 및 변수 생성__

In [7]:
# 불용어 정의
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다','ㅋㅋ']

In [8]:
okt = Okt()

tokenized_data = []
for sentence in tqdm(new_df['comments']):
    tokenized_sentence = okt.morphs(sentence, norm=True, stem=True)  # 토큰화
    stopwords_removed_sentence = [word for word in tokenized_sentence if not word in stopwords] # 불용어 제거
    tokenized_data.append(stopwords_removed_sentence)

100%|██████████| 198/198 [01:07<00:00,  2.91it/s]


In [9]:
tokenized_df = pd.DataFrame([])
tokenized_df["token_final"] = tokenized_data
tokenized_df

,token_final
0,"[교수, 님, 정말, 재미있다, 강의, 해주다, 자주, 뵈다, 좋다, 김기범, 교수..."
1,"[OCN, 켜다, 김, 왕, 까지에, 서다, 허준, 씨, 처음, 보다, 때, 이렇게..."
2,"[교수, 님, 강의, 이렇게, 쉬다, 들다, 수, 있다, 행운, 이고, 감사하다, ..."
3,"[영화, 크리에이터, 예매, 권, 이벤트, 영상, 영화, 대한, 기, 대, 평, 댓..."
4,"[우와, 영상, 재밌다, 보다, 넘다, 넘다, 차분하다, 전달, 력, 있다, 말씀,..."
...,...
193,"[다양하다, 천문학, 이야기, 전, 수, 있다, 좋다, 기회, 주신, BODA, 분..."
194,"[곧, 안전하다, 값싸다, 우주, 여행, 실현, 되다, 바라다, 우주, 여행, 관련..."
195,"[오늘, 초대, 주다, 감사하다, 자주, 놀러와, 서, 축구, 이야기, 해보다, 영..."
196,"[여자, 위해, 여자, 가장, 자다, 알다, 산부인과, 의사, 언니, 김지연, 원장..."


In [10]:
new_df["views"]

0       194914
1       319933
3        98384
4       402196
5       150218
        ...   
195     287723
196    2298575
197     362675
198     353262
199     744042
Name: views, Length: 198, dtype: object

In [11]:
tokenized_df = pd.DataFrame([])
tokenized_df["token_final"] = tokenized_data
tokenized_df["views"] = new_df.views
# tokenized_df["views"] = tokenized_df["views"].fillna(tokenized_df['views'].mean()) 
tokenized_df.dropna(inplace=True)
tokenized_df.head(5)

,token_final,views
0,"[교수, 님, 정말, 재미있다, 강의, 해주다, 자주, 뵈다, 좋다, 김기범, 교수...",194914
1,"[OCN, 켜다, 김, 왕, 까지에, 서다, 허준, 씨, 처음, 보다, 때, 이렇게...",319933
3,"[영화, 크리에이터, 예매, 권, 이벤트, 영상, 영화, 대한, 기, 대, 평, 댓...",98384
4,"[우와, 영상, 재밌다, 보다, 넘다, 넘다, 차분하다, 전달, 력, 있다, 말씀,...",402196
5,"[BODA, X, 우주먼지, 콜라보, 티셔츠, 사르다, https, smartsto...",150218


In [12]:
### ============ 기존 코드 (비교용) ============
#df = pd.DataFrame([])
#df["token_noun"] = new_df.comments.apply(okt.nouns)
#df["token_morphs"] = new_df.comments.apply(okt.morphs)
#df["label"] = new_df.label
# ============ 저장 ================
#df.to_csv("origin_noun.csv", index=False, encoding='utf-8-sig')
#df.to_csv("origin_morphs.csv", index=False, encoding='utf-8-sig')
tokenized_df.to_csv("tokenized_df_reg.csv", index=False, encoding='utf-8-sig')

> __2.4 단어 임베딩__

In [13]:
embedding_model = Word2Vec(tokenized_df['token_final'], 
                           sg = 2, # skip-gram
                           vector_size = 100, 
                           window = 2, 
                           min_count = 1, 
                           workers = 8
                           )

print(embedding_model) 

Word2Vec<vocab=18062, vector_size=100, alpha=0.025>


In [14]:
embedding_model.wv.save_word2vec_format('week3pj_tokens_eng_w2v_reg')
loaded_model = KeyedVectors.load_word2vec_format('week3pj_tokens_eng_w2v_reg') # 모델 로드

model_result = loaded_model.most_similar("AI")
print(model_result)

[('미생물', 0.9698936343193054), ('방향', 0.9662998914718628), ('음모론', 0.96424800157547), ('길', 0.9633345603942871), ('초전도체', 0.9623009562492371), ('귀신', 0.9619728326797485), ('미래', 0.9615159630775452), ('평생', 0.9613578915596008), ('주변', 0.9612826704978943), ('다만', 0.9611051678657532)]


## __3. 모델 학습__
- 일단 테스트 모델은 임시로 Classification 문제로 정의함 

In [15]:
torch.cuda.empty_cache()

> __3.0 split data - train/val__

In [16]:
df = pd.read_csv("tokenized_df_reg.csv")
#df = df.drop(labels="Unnamed: 0", axis=1)
df

,token_final,views
0,"['교수', '님', '정말', '재미있다', '강의', '해주다', '자주', '...",194914
1,"['OCN', '켜다', '김', '왕', '까지에', '서다', '허준', '씨'...",319933
2,"['영화', '크리에이터', '예매', '권', '이벤트', '영상', '영화', ...",98384
3,"['우와', '영상', '재밌다', '보다', '넘다', '넘다', '차분하다', ...",402196
4,"['BODA', 'X', '우주먼지', '콜라보', '티셔츠', '사르다', 'ht...",150218
...,...,...
191,"['다양하다', '천문학', '이야기', '전', '수', '있다', '좋다', '...",132143
192,"['곧', '안전하다', '값싸다', '우주', '여행', '실현', '되다', '...",758138
193,"['오늘', '초대', '주다', '감사하다', '자주', '놀러와', '서', '...",287723
194,"['여자', '위해', '여자', '가장', '자다', '알다', '산부인과', '...",2298575


In [17]:
rng = RandomState()

tr = df.sample(frac=0.8, random_state=rng)
val = df.loc[~df.index.isin(tr.index)]

tr.to_csv('train.csv', index=False, encoding='utf-8-sig')
val.to_csv('validation.csv', index=False, encoding='utf-8-sig')

> __3.1 실험 설계__

In [18]:
import torchtext
from torchtext.legacy.data import Field 
# from torchtext.data import Field

def tokenizer(text):
    text = re.sub('[\[\]\']', '', str(text))
    text = text.split(', ')
    return text

TEXT = Field(tokenize=tokenizer)
LABEL = Field(sequential = False)

In [19]:
from torchtext.legacy.data import TabularDataset 
# from torchtext.data import TabularDataset 

train, validation = TabularDataset.splits(
    path = './',
    train = 'train.csv',
    validation = 'validation.csv',
    format = 'csv',
    fields = [('text', TEXT), ('label', LABEL)],
    skip_header = True
)

print("Train:", train[0].text,  train[0].label)
print("Validation:", validation[0].text, validation[0].label)

Train: ['저', '육군', '2002년', '도에', '제대', '차다', '대단하다', '나라', '지키다', '지키다', '분', '다', '존경', '받다', '초', '급', '것', '중', '급', '것', '중요', '치', '않다', '나라', '위해', '훈련', '받다', '모든', '특수부대', '원', '존경', '하고', '감사하다', '부', '199', '기', '예비역', '음', '탐사', '이며', '근무', '지역', '2', '함대', '전남', '함', '제', '천', '함', '에서', '3년', '5', '개월', '있다', '제', '기수', '전문', '대학', '장학생', '기수', '이며', 'SSU', 'UDT', 'SEAL', 'EOD', '포함', '까지', '있다', '보다', 'UDT', '동기', '을', '감안', '훈련', '조금', '더', '힘들다', '부분', '있다', '저', '기초', '훈련', '끝나다', '후반기', '교육', '을', '기병', '교', '나중', '에는', '정통', '교', '로', '바뀌다', '로', '가다', '많이', '편하다', 'UDT', '동기', '오히려', '후반기', '부터', '본격', '적', '인', '훈련', '시작', '되어다', '후반기', '교육', '받다', '교내', '생활', '때', '밤', '면', '진기', '사', '쪽', '붙다', '특수전', '학교', '에서', '울리다', '비명', '가깝다', '훈련', '고함', '소리', '로', '항상', '듣기', '생식', '주', '훈련', '기본', '이며', '하수', '구', '기다', '훈련', '포박', '잠수', '유영', '훈련', '을', '보다', '정말', '대단하다', '유디티', '훈련', '보', '다니다', '힘들다', '고', '되다', '훈련', '특수전', '요원', '양성', '과정', '이군', '요', '자랑스럽다', '영상

In [21]:
import torch
from torchtext.vocab import Vectors
from torchtext.legacy.data import BucketIterator 
# from torchtext.data import BucketIterator 

vectors = Vectors(name="week3pj_tokens_eng_w2v_reg")

TEXT.build_vocab(train, vectors = vectors, min_freq = 1, max_size = None)
LABEL.build_vocab(train)

vocab = TEXT.vocab

# device = "cpu" 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_iter, validation_iter = BucketIterator.splits(
    datasets = (train, validation),
    batch_size = 8,
    device = device,
    sort = False
)

print('임베딩 벡터의 개수와 차원 : {} '.format(TEXT.vocab.vectors.shape))

100%|██████████| 18062/18062 [00:00<00:00, 27839.60it/s]


임베딩 벡터의 개수와 차원 : torch.Size([16084, 100]) 


> __3.2 TextCNN 모델링__

In [22]:
import torch.nn as nn   
import torch.optim as optim 
import torch.nn.functional as F 

class TextCNN(nn.Module): 
    
    def __init__(self, vocab_built, emb_dim, dim_channel, kernel_wins):
        
        super(TextCNN, self).__init__()
        
        self.embed = nn.Embedding(len(vocab_built), emb_dim)
        self.embed.weight.data.copy_(vocab_built.vectors)      
    
        self.convs = nn.ModuleList([nn.Conv2d(1, dim_channel, (w, emb_dim)) for w in kernel_wins])
        self.relu = nn.ReLU()                
        self.dropout = nn.Dropout(0.4)         
        self.fc = nn.Linear(len(kernel_wins)*dim_channel, 1)     
        
    def forward(self, x):  
      
        emb_x = self.embed(x)           
        emb_x = emb_x.unsqueeze(1)  

        con_x = [self.relu(conv(emb_x)) for conv in self.convs]       

        pool_x = [F.max_pool1d(x.squeeze(-1), x.size()[2]) for x in con_x]    
        
        fc_x = torch.cat(pool_x, dim=1) 
        fc_x = fc_x.squeeze(-1)       
        fc_x = self.dropout(fc_x)         

        logit = self.fc(fc_x)     
        
        return logit

> __3.3 모델 train/eavluate 함수 정의__

In [23]:
def train(model, device, train_itr, optimizer):
    
    model.train()                               
    ae, train_loss = 0.0,0        
    
    for batch in train_itr:
        
        text, target = batch.text, batch.label      
        text = torch.transpose(text, 0, 1)          
        target.data.sub_(1)                                 
        text, target = text.to(device), target.to(device)  

        optimizer.zero_grad()                           
        logit = model(text)                         
    
        loss = F.cross_entropy(logit, target)   
        loss.backward()  
        optimizer.step()  
        
        train_loss += loss.item()    
        # result = torch.max(logit,1)[1] 
        ae += torch.sum(torch.abs(logit))
        
    train_loss /= len(train_itr.dataset)
    mae = ae / len(train_itr.dataset)

    return train_loss, mae

In [24]:
def evaluate(model, device, itr):
    
    model.eval()
    ae, test_loss = 0.0, 0

    for batch in itr:
        
        text = batch.text
        target = batch.label
        
        text = torch.transpose(text, 0, 1)
        target.data.sub_(1)
        text, target = text.to(device), target.to(device)
        
        logit = model(text)
        loss = F.cross_entropy(logit, target)
        #print(target,logit)
        test_loss += loss.item()
        # result = torch.max(logit,1)[1]
        ae += torch.sum(torch.abs(logit))

    test_loss /= len(itr.dataset) 
    mae = ae / len(itr.dataset)
    
    return test_loss, mae

> __3.4 모델 학습 및 성능 확인__

In [25]:
EPOCH = 15

In [29]:
model = TextCNN(vocab, 100, 10, [1,1,1]).to(device)
print(model)

optimizer = optim.Adam(model.parameters(), lr=0.001)

best_test_mae = -1

for epoch in range(1, EPOCH+1):
 
    tr_loss, tr_mae = train(model, device, train_iter, optimizer) 
    print('Train Epoch: {} \t Loss: {} \t MAE: {}%'.format(epoch, tr_loss, tr_mae))
    
    val_loss, val_mae = evaluate(model, device, validation_iter)
    print('Valid Epoch: {} \t Loss: {} \t MAE: {}%'.format(epoch, val_loss, val_mae))
        
    if val_mae > best_test_mae:
        best_test_mae = val_mae
        
        print("model saves at {} MAE".format(best_test_mae))
        torch.save(model.state_dict(), "TextCNN_Best_Validation")
    
    print('-----------------------------------------------------------------------------')

TextCNN(
  (embed): Embedding(16084, 100)
  (convs): ModuleList(
    (0): Conv2d(1, 10, kernel_size=(1, 100), stride=(1, 1))
    (1): Conv2d(1, 10, kernel_size=(1, 100), stride=(1, 1))
    (2): Conv2d(1, 10, kernel_size=(1, 100), stride=(1, 1))
  )
  (relu): ReLU()
  (dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=30, out_features=1, bias=True)
)


IndexError: Target 14 is out of bounds.